<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/jc/3_make_errors/e3_make_errors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install kiwipiepy
# !pip install nltk
# !pip install
# !pip install selenium

# #코랩에서 구글드라이버 연동
# !apt-get update
# !apt install chromium-chromedriver

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

Mounted at /content/drive


**아이디어 기록**

1. Critical

2. Major
    - Mis-Translation
        - "ailen" -> 외계인으로 처리

# 0.Data Load

In [2]:
import pandas as pd
import numpy as np
import json
import os
import re
import copy
from tqdm.notebook import tqdm

if os.path.exists(r"G:\내 드라이브\KB인턴관련"):
    gdrive_path = r"G:\내 드라이브\KB인턴관련"
else:
    gdrive_path = "/content/drive/MyDrive/KB인턴관련"

In [3]:
df = pd.read_csv(os.path.join(gdrive_path, "preprocessing/fin.csv"), encoding="euc-kr", index_col=0)
df.head()

,category,lawName,ENG,KOR
0,헌법,개성공업지구 지원에 관한 법률,The purpose of this Act is to enhance inter-Ko...,이 법은 개성공업지구의 개발·운영의 지원 및 개성공업지구에 투자하거나 출입·체류하는...
2,헌법,개성공업지구 지원에 관한 법률,"The term ""Gaeseong Industrial Complex"" or ""GIC...","""개성공업지구""란 남한과 북한 사이의 합의에 따라서 북한의 개성지역 일대에서 개발·..."
3,헌법,개성공업지구 지원에 관한 법률,"The term ""GIC developer"" means a South Korean ...","""개성공업지구 개발업자""란 「남북교류협력에 관한 법률」에 따른 협력사업의 승인을 받..."
4,헌법,개성공업지구 지원에 관한 법률,"The term ""GIC management organ"" means a corpor...","""개성공업지구 관리기관""이란 개성공업지구의 관리·운영을 위하여 북한의 「개성공업지구..."
5,헌법,개성공업지구 지원에 관한 법률,"The term ""local enterprise of the GIC"" means a...","""개성공업지구 현지기업""이란 남한주민이 「남북교류협력에 관한 법률」에 따른 협력사업..."


# 1.Data Fiiltering

In [4]:
keyword_idx = df.loc[df.lawName.str.contains(r"은행|금융|보험|예금|적금|대출|외환|통화|화폐|환율|예대")].index

In [5]:
tgk_idx = df.loc[df.category=="통화ㆍ국채ㆍ금융"].index

In [6]:
print(f"키워드: {len(keyword_idx)}\t통화/국채/금융: {len(tgk_idx)}")
print(f"키워드-통국금: {len(list(set(keyword_idx)-set(tgk_idx)))}")
print(f"통국금-키워드: {len(list(set(tgk_idx)-set(keyword_idx)))}")
print(f"전체:  {len(list(set(keyword_idx).union(set(tgk_idx))))}")

키워드: 16712	통화/국채/금융: 16004
키워드-통국금: 4681
통국금-키워드: 3973
전체:  20685


In [7]:
df_samp = df.loc[list(set(keyword_idx).union(set(tgk_idx)))].drop(index=[403163], axis=0)
df_samp

,category,lawName,ENG,KOR
547121,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the average monthly remuneration of each...,"예술인[「고용보험법」 제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술..."
547123,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the amount of monthly remuneration of ea...,"노무제공자[「고용보험법」 제77조의6제2항제2호 단서에 따른 단기노무제공자(이하 ""..."
547124,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,"""Where any ground prescribed by Presidential D...","법 제3조제1항에서 ""대통령령으로 정하는 사유에 해당하는 경우""란 다음 각 호의 어..."
547125,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,The standard remuneration under Article 3 (2) ...,법 제3조제2항(법 제48조의2제8항제2호 및 제48조의3제8항제2호에서 준용하는 ...
547126,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where an artist falls under any of the followi...,예술인이 다음 각 호의 어느 하나에 해당하는 경우 법 제48조의2제8항제2호에서 준...
...,...,...,...,...
78005,통화ㆍ국채ㆍ금융,국채법,Where necessary to properly issue and manage S...,기획재정부장관은 국채의 발행과 관리를 적절하게 하기 위하여 필요한 경우에는 관계 중...
78006,통화ㆍ국채ㆍ금융,국채법,Any person requested to provide materials rela...,제1항에 따라 국채 관련 자료의 제출을 요청받은 자는 특별한 사유가 없으면 요청에 ...
78007,통화ㆍ국채ㆍ금융,국채법,Where necessary to formulate and carry out pol...,기획재정부장관은 국채시장에 관한 정책의 수립·운영에 필요한 경우에는 관계 중앙행정기...
406361,통화ㆍ국채ㆍ금융,신용협동조합법 시행령,"Notwithstanding paragraph (3), a workplace cre...",제3항의 규정에 불구하고 제12조제1항제2호의 직장조합 및 동항제3호 라목의 단체조...


In [8]:
df_samp["curHyp"] = df_samp.KOR.values
df_samp["errors"] = [[] for _ in range(df_samp.shape[0])]
df_samp["tagLog"] = [dict() for _ in range(df_samp.shape[0])]
# df_samp.iloc[:, 2:4]

**주요 함수**

In [517]:
## 주요한 함수
def remake_df():
    result = df.loc[list(set(keyword_idx).union(set(tgk_idx)))]
    result["curHyp"] = result.KOR.values
    result["errors"] = [[] for _ in range(result.shape[0])]
    result["tag_log"] = [dict() for _ in range(result.shape[0])]
    return result.drop(index=[403163], axis=0)

def random_df(dfs, ratio, seed):
    np.random.seed(seed)
    return dfs.loc[np.random.choice(dfs.index.to_list(), int(dfs.shape[0]*ratio), False), :]

def random_index(srs, ratio, seed):
    np.random.seed(seed)
    return np.random.choice(srs.index, int(srs.shape[0]*ratio), False)

def copy_df(data, col=["errors", "tagLog"]):
    result = data.drop(columns=col, axis=1).copy(deep=True)
    # print(id(result), id(data))
    for c in col:
        result[c] = data[c].apply(lambda x: copy.deepcopy(x)).copy(deep=True)
    return result

def count_tag(tokens, tags=None):
    token_dict = dict()
    for token in tokens:
        if tags!=None and token.tag not in tags: continue
        try:
            token_dict[token.tag].append(token.form)
        except:
            token_dict[token.tag] = [token.form]

    return token_dict

def find_tokens(tokens, tag):
    result_tokens = []
    for token in tokens:
        if token.tag == tag:
            result_tokens.append((token.form, token.start, token.end))
    return result_tokens

def print_df(df, idx=0, end=0):
    print(df.shape[0])
    if end==0:
        values = df.index[idx:]
    else:
        values = df.index[idx:end]
    for i in values:
        print(f"_________________ {i} _________________")
        print(df.loc[i, "curHyp"])
        print(df.loc[i, "errors"])
        print(df.loc[i, "tagLog"])

# result.loc[i, "errors"].append({"severity":"major", "start_idx":start, "text":del_unit, "type":"omission"})
def input_errors(df, index, severity, start_idx, text, type):
    df.loc[index, "errors"].append({"severity":severity, "start_idx":start_idx, "text":text, "type":type})

def convert_df(df, diff_df):
    cdf = copy_df(df)
    for i in diff_df.index:
        cdf.loc[i, : ] = diff_df.loc[i, : ]
    return cdf

def find_severity(df, sv):
    df.errors.apply()

# 2.품사 태깅

In [10]:
from kiwipiepy import Kiwi

kiwi = Kiwi(num_workers=0, model_type='sbg')
kiwi_tokens = df_samp.KOR.apply(lambda x: kiwi.tokenize(x))
kiwi_tokens

547121    [(예술인, NNG, 0, 3), ([, SSO, 3, 1), (「, SSO, 4,...
547123    [(노무, NNG, 0, 2), (제공자, NNG, 2, 3), ([, SSO, 5...
547124    [(법, NNG, 0, 1), (제, XPN, 2, 1), (3, SN, 3, 1)...
547125    [(법, NNG, 0, 1), (제, XPN, 2, 1), (3, SN, 3, 1)...
547126    [(예술, NNG, 0, 2), (이, VCP, 2, 1), (ᆫ, ETM, 2, ...
                                ...                        
78005     [(기획, NNG, 0, 2), (재정부, NNG, 2, 3), (장관, NNG, ...
78006     [(제, XPN, 0, 1), (1, SN, 1, 1), (항, NNG, 2, 1)...
78007     [(기획, NNG, 0, 2), (재정부, NNG, 2, 3), (장관, NNG, ...
406361    [(제, XPN, 0, 1), (3, SN, 1, 1), (항, NNG, 2, 1)...
406362    [(조합, NNG, 0, 2), (이, JKS, 2, 1), (제, XPN, 4, ...
Name: KOR, Length: 20684, dtype: object

In [518]:
count_tokens = kiwi_tokens.apply(count_tag)
count_tokens

547121    {'NNG': ['예술인', '항', '단서', '단기', '예술인', '이하', ...
547123    {'NNG': ['노무', '제공자', '항', '단서', '단기', '노무', '...
547124    {'NNG': ['법', '항', '대통령령', '사유', '해당', '경우', '...
547125    {'NNG': ['법', '항', '법', '항', '항', '준용', '경우', ...
547126    {'NNG': ['예술', '다음', '호의', '해당', '경우', '법', '항...
                                ...                        
78005     {'NNG': ['기획', '재정부', '장관', '국채', '발행', '관리', ...
78006     {'XPN': ['제'], 'SN': ['1'], 'NNG': ['항', '국채',...
78007     {'NNG': ['기획', '재정부', '장관', '국채', '시장', '정책', ...
406361    {'XPN': ['제', '제', '제', '제', '제'], 'SN': ['3',...
406362    {'NNG': ['조합', '항', '규정', '복지', '사업', '실시', '경...
Name: KOR, Length: 20684, dtype: object

In [12]:
nng_dict = dict()
for tokens in count_tokens.apply(lambda x: x["NNG"]):
    for t in tokens:
        try:
            nng_dict[t] += 1
        except:
            nng_dict[t] = 1
len(nng_dict.keys())

3703

In [13]:
nnp_info = kiwi_tokens.apply(find_nnp_tokens)
nnp_info

547121    [(고용보험법, 5, 10)]
547123    [(고용보험법, 7, 12)]
547124                  []
547125                  []
547126                  []
                ...       
78005     [(한국은행, 56, 60)]
78006                   []
78007                   []
406361                  []
406362                  []
Name: KOR, Length: 20684, dtype: object

# 3.Error 생성

## 3.1.Critical

### 3.1.1.Critical - Omission

In [478]:
cr_om_index = random_index(df_samp.curHyp, 0.03, 10)
# print_df(df_samp.loc[cr_om_index])

In [479]:
def critical_omission(df):
    result = copy_df(df)
    random_ratio = [3/5, 2/3, 5/8, 5/7]
    np.random.seed(10)
    split_sentences = result.curHyp.apply(lambda x: x.split(" "))
    len_element = split_sentences.apply(len)
    crop_element = len_element.apply(lambda x: int(x*(np.random.choice(random_ratio, 1))))
    start_idx = (len_element-crop_element).apply(lambda x: np.random.choice([i for i in range(x)], 1)[0])
    end_idx = start_idx+crop_element

    for i in df.index:
        reduce_sentence = " ".join([split_sentences[i][k] for k in range(start_idx[i], end_idx[i]+1)])
        result.loc[i, "tagLog"]["omission"] = reduce_sentence
        cur_hyp = result.loc[i, "curHyp"]
        sidx = cur_hyp.find(reduce_sentence)

        conv_hyp = " ".join([split_sentences[i][j] for j in range(len_element[i]) if (start_idx[i] > j) or (end_idx[i] < j)])
        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, "critical", sidx, reduce_sentence, "omission")

    del split_sentences, len_element, crop_element, start_idx, end_idx
    return result
df_crom = convert_df(df_samp, critical_omission(df_samp.loc[cr_om_index]))

In [480]:
df_crom.loc[cr_om_index].shape

(620, 7)

### 3.1.2.Mis-Translation

In [481]:
# cr_mt_index = random_index(df_crom, 0.03, 111)
# temp = df_crom.loc[cr_mt_index]
# temp.loc[temp.errors.apply(len)>0].iloc[:, 3:]

## 3.2.Major

### 3.2.1.Major - Omission

**고유명사 생략**

In [482]:
nnp_info = kiwi_tokens.apply(lambda x: find_tokens(x, "NNP")).drop(index=cr_om_index, axis=0)
nnp_info

547121    [(고용보험법, 5, 10)]
547123    [(고용보험법, 7, 12)]
547124                  []
547125                  []
547126                  []
                ...       
78005     [(한국은행, 56, 60)]
78006                   []
78007                   []
406361                  []
406362                  []
Name: KOR, Length: 20064, dtype: object

In [483]:
np.random.seed(20)
change_val = nnp_info[nnp_info.apply(len) > 0].apply(lambda x: x[np.random.choice(len(x), 1)[0]])
ch6 = change_val[change_val.apply(lambda x: len(x[0])) > 3]
chv_index = random_index(ch6, 0.5, 20)
drop_idx = ch6.loc[ch6.apply(lambda x: len(x[0].split("·")))>1].index.to_list()
ch6[chv_index]

407195         (전자서명, 90, 94)
401241       (농업협동조합, 38, 44)
419950      (전기통신금융사기, 5, 13)
577602         (한국은행, 55, 59)
414008         (기업재무, 30, 34)
                 ...         
396635         (예금보험공사, 0, 6)
403404       (상호저축은행, 43, 49)
403140    (상호저축은행중앙회, 68, 77)
419862     (전기통신금융사기, 25, 33)
421861           (한국은행, 0, 4)
Name: KOR, Length: 1185, dtype: object

In [484]:
ch6.loc[ch6.apply(lambda x: len(x[0].split("·")))>1]    # 시리즈에 검출되는 고유명사가 있다면 drop 시킬 것!

Series([], Name: KOR, dtype: object)

In [333]:
# i = 80167
# del_word, start, end = ch6.loc[i]
# cur_hyp = df_crom.loc[i, "curHyp"]

# forward = cur_hyp[:start]
# backward = cur_hyp[start:]
# split_back = backward.split(" ")
# del_unit = split_back[0]
# print(cur_hyp[end])
# if cur_hyp[start-1] == '"' or cur_hyp[start-1]=='「' or cur_hyp[end]=="」" or cur_hyp[end]=='\"' :
#     pattern = f'\"[^\"]*{del_word}[^\"]*\"|「[^」]*{del_word}[^」]*」'
#     find_comp = re.compile(pattern)
#     find_res = find_comp.search(cur_hyp)
#     start, end = find_res.span()
#     del_word = cur_hyp[start:end]
#     while cur_hyp[end]!=" " and  end<len(cur_hyp):
#         end+=1
#     while cur_hyp[start]!=" " and start>0:
#         start-=1
#     del_unit = cur_hyp[start:end]
#     conv_hyp = cur_hyp[:start].strip()+" "+cur_hyp[end:].strip()
# elif "(" in del_unit:
#     pattern = f'{del_word}[^\)]+\)'
#     find_comp = re.compile(pattern)
#     find_res = find_comp.search(cur_hyp)
#     start, end = find_res.span()
#     while cur_hyp[end]!=" " and  end<len(cur_hyp):
#         end+=1
#     while cur_hyp[start]!=" " and start>0:
#         start-=1

#     del_unit = cur_hyp[start:end]
#     conv_hyp = cur_hyp[:start].strip()+" "+cur_hyp[end:].strip()

# print(del_word, start, end)
# print(del_unit)
# print(conv_hyp)

In [334]:
# re.sub(pattern, "", cur_hyp)

In [485]:
def major_omission(ch_srs, df_local):
    result = copy_df(df_local)
    for i in ch_srs.index:
        del_word, start, end = ch_srs.loc[i]
        cur_hyp = result.loc[i, "curHyp"]

        forward = cur_hyp[:start]
        backward = cur_hyp[start:]
        split_back = backward.split(" ")
        del_unit = split_back[0]
        back_mod = " ".join(split_back[1:])

        try:
            if start==0:
                conv_hyp = (forward+back_mod).strip(" ")
            elif cur_hyp[start-1] == '"' or cur_hyp[start-1]=='「' or cur_hyp[end]=="」" or cur_hyp[end]=='\"' :
                pattern = f'\"[^\"]*{del_word}[^\"]*\"|「[^」]*{del_word}[^」]*」'
                find_comp = re.compile(pattern)
                find_res = find_comp.search(cur_hyp)
                start, end = find_res.span()
                while cur_hyp[end]!=" " and  end<len(cur_hyp):
                    end+=1
                while cur_hyp[start]!=" " and start>0:
                    start-=1
                del_unit = cur_hyp[start:end]
                conv_hyp = cur_hyp[:start].strip()+" "+cur_hyp[end:].strip()
            elif "(" in del_unit:
                pattern = f'{del_word}[^\)]+\)'
                find_comp = re.compile(pattern)
                find_res = find_comp.search(cur_hyp)
                start, end = find_res.span()
                while cur_hyp[end]!=" " and  end<len(cur_hyp):
                    end+=1
                while cur_hyp[start]!=" " and start>0:
                    start-=1
                del_unit = cur_hyp[start:end]
                conv_hyp = cur_hyp[:start].strip()+" "+cur_hyp[end:].strip()
            else:
                conv_hyp = (forward+back_mod).strip(" ")
            result.loc[i, "tagLog"]["omission"] = del_unit
            result.loc[i, "curHyp"] = conv_hyp
            input_errors(result, i, "major", start, del_unit, "omission")
        except:
            pass

    return result

In [486]:
df_maom = convert_df(df_crom, major_omission(ch6[chv_index], df_crom.loc[chv_index, :]))
# df_maom.loc[chv_index]

In [487]:
ch6[555553]

('근로복지공단', 52, 58)

In [489]:
# aaa = 53
# df_maom.loc[chv_index].iloc[aaa:aaa+10, 3:]
df_maom.loc[chv_index].shape

(1185, 7)

In [490]:
df_maom.loc[df_maom.errors.apply(len)==0].shape[0]

18879

**일반 명사 생략**

In [491]:
nng_info = kiwi_tokens.apply(lambda x: find_tokens(x, "NNG")).drop(index=df_maom.loc[df_maom.errors.apply(len)>0].index, axis=0)
nng_info

547121    [(예술인, 0, 3), (항, 20, 21), (단서, 25, 27), (단기, ...
547123    [(노무, 0, 2), (제공자, 2, 5), (항, 22, 23), (단서, 27...
547124    [(법, 0, 1), (항, 7, 8), (대통령령, 12, 16), (사유, 23...
547125    [(법, 0, 1), (항, 7, 8), (법, 9, 10), (항, 19, 20)...
547126    [(예술, 0, 2), (다음, 5, 7), (호의, 10, 12), (해당, 20...
                                ...                        
78005     [(기획, 0, 2), (재정부, 2, 5), (장관, 5, 7), (국채, 9, ...
78006     [(항, 2, 3), (국채, 8, 10), (관련, 11, 13), (자료, 14...
78007     [(기획, 0, 2), (재정부, 2, 5), (장관, 5, 7), (국채, 9, ...
406361    [(항, 2, 3), (규정, 5, 7), (항, 20, 21), (직장, 26, ...
406362    [(조합, 0, 2), (항, 6, 7), (규정, 9, 11), (복지, 16, ...
Name: KOR, Length: 18879, dtype: object

In [492]:
np.random.seed(36)
change_val = nng_info[nng_info.apply(len) > 0].apply(lambda x: x[np.random.choice(len(x), 1)[0]])
ch6 = change_val[change_val.apply(lambda x: len(x[0])) > 2]
chv_index = random_index(ch6, 0.55, 36)
drop_idx = ch6.loc[ch6.apply(lambda x: len(x[0].split("·")))>1].index.to_list()
ch6[chv_index]

420293     (발행인, 9, 12)
403506      (위원회, 2, 5)
401537    (대리점, 34, 37)
402059      (위원회, 2, 5)
398492    (책임자, 20, 23)
              ...      
403133      (연합회, 1, 4)
85903     (수출입, 25, 28)
259696    (감독원, 51, 54)
398562      (임직원, 6, 9)
402058    (자본금, 32, 35)
Name: KOR, Length: 1088, dtype: object

In [493]:
df_mo = convert_df(df_maom, major_omission(ch6[chv_index], df_maom.loc[chv_index, :]))

In [494]:
df_mo.loc[df_mo.errors.apply(len)>0].shape[0]

2888

In [495]:
df_mo.loc[chv_index]

,category,lawName,ENG,KOR,curHyp,errors,tagLog
420293,통화ㆍ국채ㆍ금융,주식·사채 등의 전자등록에 관한 법률,"An issuer of electronically registered stocks,...",전자등록주식등의 발행인은 제1항에 따른 전자등록기관을 통한 권리 행사를 위하여 대통...,전자등록주식등의 제1항에 따른 전자등록기관을 통한 권리 행사를 위하여 대통령령으로 ...,"[{'severity': 'major', 'start_idx': 9, 'text':...",{'omission': '발행인은'}
403506,통화ㆍ국채ㆍ금융,상호저축은행법 시행령,Standards for liquidity determined by the Fina...,금융위원회가 법 제22조의2제1항제5호에 따라 정하는 유동성 기준에는 다음 각 호의...,금융법 제22조의2제1항제5호에 따라 정하는 유동성 기준에는 다음 각 호의 사항이 ...,"[{'severity': 'major', 'start_idx': 2, 'text':...",{'omission': '위원회가'}
401537,통화ㆍ국채ㆍ금융,보험업법 시행령,Any of the following entities among insurance ...,"법 제83조제2항에 따라 법 제91조제2항에 따른 금융기관보험대리점등(이하 ""금융기...",법 제83조제2항에 따라 법 제91조제2항에 따른 중 다음 각 호의 어느 하나에 해...,"[{'severity': 'major', 'start_idx': 27, 'text'...","{'omission': ' 금융기관보험대리점등(이하 ""금융기관보험대리점등""이라 한다)'}"
402059,통화ㆍ국채ㆍ금융,보험업법 시행령,The Financial Services Commission may determin...,금융위원회는 제1항부터 제3항까지의 규정에 관하여 필요한 세부 기준을 정하여 고시할...,금융제1항부터 제3항까지의 규정에 관하여 필요한 세부 기준을 정하여 고시할 수 있다.,"[{'severity': 'major', 'start_idx': 2, 'text':...",{'omission': '위원회는'}
398492,통화ㆍ국채ㆍ금융,금융회사의 지배구조에 관한 법률,If an executive officer of a financial company...,금융위원회는 금융회사의 직원(업무집행책임자를 포함한다. 이하 이 조에서 같다)이 별...,금융위원회는 금융회사의 직원(업무집행포함한다. 이하 이 조에서 같다)이 별표 각 호...,"[{'severity': 'major', 'start_idx': 20, 'text'...",{'omission': '책임자를'}
...,...,...,...,...,...,...,...
403133,통화ㆍ국채ㆍ금융,상호저축은행법,(Applicability to Executive officers of Federa...,(연합회 임원에 관한 적용례) 제25조의4의 개정규정은 이 법 시행후 최초로 선출하...,(임원에 관한 적용례) 제25조의4의 개정규정은 이 법 시행후 최초로 선출하는 연합...,"[{'severity': 'major', 'start_idx': 1, 'text':...",{'omission': '연합회'}
85903,통화ㆍ국채ㆍ금융,한국수출입은행법,The board of directors shall be comprised of t...,"이사회는 은행장·전무이사와 이사로 구성하고, 수출입은행의 업무에 관한 중요 사항을 ...","이사회는 은행장·전무이사와 이사로 구성하고, 업무에 관한 중요 사항을 의결한다.","[{'severity': 'major', 'start_idx': 25, 'text'...",{'omission': '수출입은행의'}
259696,형사법,보험사기방지 특별법,The Financial Services Commission may partiall...,금융위원회는 필요한 경우에는 이 법에 따른 권한의 일부를 대통령령으로 정하는 바에 ...,금융위원회는 필요한 경우에는 이 법에 따른 권한의 일부를 대통령령으로 정하는 바에 ...,"[{'severity': 'major', 'start_idx': 51, 'text'...",{'omission': '감독원의'}
398562,통화ㆍ국채ㆍ금융,금융회사의 지배구조에 관한 법률,Any executive officer or an employee of a fina...,금융회사의 임직원으로서 이 법에 따른 서류의 비치·제출·보고·공고 또는 공시를 게을...,금융회사의 이 법에 따른 서류의 비치·제출·보고·공고 또는 공시를 게을리한 자에게는...,"[{'severity': 'major', 'start_idx': 6, 'text':...",{'omission': '임직원으로서'}


### 3.2.2.Major-Addition

In [496]:
ny_idx = df_mo.loc[df_mo.errors.apply(len)==0].index
kiwi_tokens[ny_idx]

547121    [(예술인, NNG, 0, 3), ([, SSO, 3, 1), (「, SSO, 4,...
547123    [(노무, NNG, 0, 2), (제공자, NNG, 2, 3), ([, SSO, 5...
547124    [(법, NNG, 0, 1), (제, XPN, 2, 1), (3, SN, 3, 1)...
547125    [(법, NNG, 0, 1), (제, XPN, 2, 1), (3, SN, 3, 1)...
547126    [(예술, NNG, 0, 2), (이, VCP, 2, 1), (ᆫ, ETM, 2, ...
                                ...                        
78005     [(기획, NNG, 0, 2), (재정부, NNG, 2, 3), (장관, NNG, ...
78006     [(제, XPN, 0, 1), (1, SN, 1, 1), (항, NNG, 2, 1)...
78007     [(기획, NNG, 0, 2), (재정부, NNG, 2, 3), (장관, NNG, ...
406361    [(제, XPN, 0, 1), (3, SN, 1, 1), (항, NNG, 2, 1)...
406362    [(조합, NNG, 0, 2), (이, JKS, 2, 1), (제, XPN, 4, ...
Name: KOR, Length: 17796, dtype: object

In [497]:
tag = "NNG"
nouns =list(set(count_tokens.apply(lambda x: x[tag] if tag in x.keys() else []).sum()))
len(nouns)

3703

In [498]:
ma_ad_index = random_index(df_mo, 0.22, 55)
df_mo.loc[ma_ad_index].shape

(4550, 7)

In [499]:
nouns_lists = count_tokens[ma_ad_index].apply(lambda x: x["NNG"])

In [500]:
nnpg_info = count_tokens[ma_ad_index].apply(lambda x: [val for val in list(set(x["NNG"])) if len(val)>1])
drop_index = nnpg_info[nnpg_info.apply(len) == 0].index.to_list()
nnpg_info.drop(index=drop_index, axis=0, inplace=True)
ma_ad_index = list(set(ma_ad_index)-set(drop_index))

In [501]:
np.random.seed(55)
add_position = nnpg_info.apply(lambda x: x[np.random.choice(range(0, len(x)))])
add_position

411178     회사
555763     요양
80092      금융
84877      자산
81274      경우
         ... 
82668      규정
406007     기금
396112    사무소
77984      제공
548401     이상
Name: KOR, Length: 4549, dtype: object

In [502]:
np.random.seed(55)
add_words = nouns_lists.apply(lambda x: np.random.choice(list(set(nouns)-set(x))))
add_words

411178      몰
555763     환불
80092     인건비
84877      유망
81274      발급
         ... 
82668      억제
406007     법인
396112     행위
77984      중재
548401    사례금
Name: KOR, Length: 4550, dtype: object

In [503]:
def major_addition(df):
    np.random.seed(55)
    result=copy_df(df)
    josa_list = ["의", "에 의한", "에", "만", "만의"]
    for i in df.index:
        cur_hyp = df.loc[i, "curHyp"]
        josa_add = np.random.choice(josa_list)
        word = add_position[i]
        start = cur_hyp.find(word)
        if word not in cur_hyp:
            continue
        add_word = add_words[i]+josa_add+" "
        # print(start, cur_hyp)
        while start>0 and cur_hyp[start]!=" ":
            start-=1
        conv_hyp = cur_hyp[:start+1]+f"{add_word}"+cur_hyp[start+1:]
        # print(add_word, word)
        # print(start, conv_hyp)
        result.loc[i, "tagLog"]["addition"] = add_word.strip()
        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, "major", start+1, add_word.strip(), "addition")
    return result

df_maad = convert_df(df_mo, major_addition(df_mo.loc[ma_ad_index]))

In [504]:
df_maad.loc[df_maad.errors.apply(len)>0].shape

(6821, 7)

## 3.3.Minor

### 3.3.1.Mis-Translation

In [512]:
js_mstr_index = random_index(df_maad, 0.2, 1010)
len(js_mstr_index)

4136

In [515]:
js_tokens = df_maad.loc[js_mstr_index].curHyp.apply(lambda x: kiwi.tokenize(x))
js_tokens

395580    [(하, VV, 1, 1), (ᆫ다, EF, 1, 2), (), SSC, 3, 1)...
416588    [(그, NP, 0, 1), (밖에, JX, 2, 2), (투자자, NNG, 5, ...
399808    [(보험, NNG, 0, 2), (회사, NNG, 2, 2), (가, JKC, 4,...
397909    [(법, NNG, 0, 1), (제, XPN, 2, 1), (48, SN, 3, 2...
547949    [(제, XPN, 0, 1), (1, SN, 1, 1), (항, NNG, 2, 1)...
                                ...                        
399674    [(법, NNG, 0, 1), (제, XPN, 2, 1), (18, SN, 3, 2...
189526    [(자영업자, NNG, 0, 4), (에, JKB, 4, 1), (대하, VV, 6...
419406    [(별표, NNG, 0, 2), (21, SN, 3, 2), (제, XPN, 6, ...
401857    [(법, NNG, 0, 1), (제, XPN, 2, 1), (106, SN, 3, ...
81056     [(신탁, NNG, 0, 2), (업자, NNG, 2, 2), (는, JX, 4, ...
Name: curHyp, Length: 4136, dtype: object

In [531]:
tags=[
    "JKS",
    "JKC",
    "JKG",
    "JKO",
    "JKB",
    "JKV",
    "JKQ",
    "JX",
    "JC"
]
js_dictionary = {"JKS":[],"JKC":[],"JKG":[],"JKO":[],"JKB":[],"JKV":[],"JKQ":[],"JX":[],"JC":[]}
js_count_token = js_tokens.apply(lambda x: count_tag(x, tags))

for i in js_mstr_index:
    for key in js_count_token[i]:
        js_dictionary[key]+=js_count_token[i][key]
        js_dictionary[key] = list(set(js_dictionary[key]))

js_dictionary

{'JKS': ['이', '가'],
 'JKC': ['이', '가'],
 'JKG': ['에', '의'],
 'JKO': ['를', '을'],
 'JKB': ['으로써',
  '에',
  '으로서',
  '서부터',
  '과',
  '서',
  '으로',
  '와',
  '로서',
  '만큼',
  '로',
  '으로부터',
  '로부터',
  '에게',
  '로써',
  '에서',
  '보다'],
 'JKV': [],
 'JKQ': ['이라'],
 'JX': ['부터',
  '만',
  '는',
  '까지',
  '마다',
  '든지',
  '다',
  '밖에',
  '이나',
  '이란',
  '거나',
  '대로',
  '란',
  '도',
  '이든',
  '은',
  'ᆫ',
  '나'],
 'JC': ['과', '이나', '하고', '와', '나']}

In [511]:
def josa_mistranslation(df):
    result = copy_df(df)

    '''
        JKS	주격 조사
        JKC	보격 조사
        JKG	관형격 조사
        JKO	목적격 조사
        JKB	부사격 조사
        JKV	호격 조사
        JKQ	인용격 조사
        JX	보조사
        JC	접속 조사
    '''
    ## 를/을

    ##

,category,lawName,ENG,KOR,curHyp,errors,tagLog
547124,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,"""Where any ground prescribed by Presidential D...","법 제3조제1항에서 ""대통령령으로 정하는 사유에 해당하는 경우""란 다음 각 호의 어...","법 제3조제1항에서 ""대통령령으로 정하는 사유에 해당하는 경우""란 다음 각 호의 어...",[],{}
547125,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,The standard remuneration under Article 3 (2) ...,법 제3조제2항(법 제48조의2제8항제2호 및 제48조의3제8항제2호에서 준용하는 ...,법 제3조제2항(법 제48조의2제8항제2호 및 제48조의3제8항제2호에서 준용하는 ...,[],{}
547126,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where an artist falls under any of the followi...,예술인이 다음 각 호의 어느 하나에 해당하는 경우 법 제48조의2제8항제2호에서 준...,예술인이 다음 각 호의 어느 하나에 해당하는 경우 법 제48조의2제8항제2호에서 준...,"[{'severity': 'major', 'start_idx': 78, 'text'...",{'addition': '잡지에 의한'}
547127,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where a worker falls under any of the followin...,노무제공자가 다음 각 호의 어느 하나에 해당하는 경우 법 제48조의3제8항제2호에서...,노무제공자가 다음 각 호의 어느 하나에 해당하는 경우 법 제48조의3제8항제2호에서...,[],{}
394152,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,The purpose of this Act is to facilitate forei...,이 법은 경제자유구역의 지정 및 운영을 통하여 외국인투자기업 및 국내복귀기업의 경영...,이 법은 경제자유구역의 지정 및 운영을 통하여 외국인투자기업 및 국내복귀기업의 경영...,[],{}
...,...,...,...,...,...,...,...
406311,통화ㆍ국채ㆍ금융,신용협동조합법 시행령,A union which has an auditor as a standing mem...,제14조제5항에 따라 감사를 상임으로 하는 조합은 같은 항에 따른 요건에 해당하게 ...,제14조제5항에 따라 감축에 감사를 상임으로 하는 조합은 같은 항에 따른 요건에 해...,"[{'severity': 'major', 'start_idx': 12, 'text'...",{'addition': '감축에'}
77998,통화ㆍ국채ㆍ금융,국채법,Where deemed necessary for maturity diversific...,기획재정부장관은 국채 상환기일의 분산이나 국채의 유동성 조절 등을 위하여 필요하다고...,기획재정부장관은 국채 상환기일의 분산이나 국채의 유동성 조절 등을 위하여 필요하다고...,[],{}
77999,통화ㆍ국채ㆍ금융,국채법,Simultaneously with the purchase of State bond...,기획재정부장관은 제1항에 따른 방법으로 국채를 매입하면서 동시에 국채를 발행하여 매...,기획재정부장관은 제1항에 따른 방법으로 국채를 매입하면서 출금에 동시에 국채를 발행...,"[{'severity': 'major', 'start_idx': 32, 'text'...",{'addition': '출금에'}
78001,통화ㆍ국채ㆍ금융,국채법,The right to receive payments of principal and...,국채의 원금 및 이자를 받을 권리는 5년간 행사하지 아니하면 시효(시효)의 완성으로...,국채의 원금 및 이자를 받을 권리는 5년간 행사하지 아니하면 시효(시효)의 완성으로...,[],{}


## 3.3.Neutral - Mis-translation(사소한 번역 오류)

In [ ]:
df_samp.loc[df_samp.curHyp.str.contains(r"달러|제곱미터|퍼센트")][["ENG", "curHyp"]]

,ENG,curHyp
394528,The amount of foreign investment to be investe...,관광사업에 투자하려는 외국인투자 금액이 미합중국화폐 5억달러 이상일 것
399090,The fees for guarantee which the management in...,관리기관이 법 제11조의 규정에 의하여 받는 보증료는 보증한 금액에 보증료율을 곱하...
399094,The penalties for breach of contract which the...,관리기관이 법 제11조의2의 규정에 의하여 징수하는 위약금은 보증한 채무중 이행되지...
77386,Where a development project under subparagraph...,"법 제6조제1항제3호의2에 따른 단위개발사업지구(이하 ""단위개발사업지구""라 한다) ..."
77583,"Notwithstanding paragraphs (1) and (2), the si...",제1항 및 제2항에도 불구하고 「주택법」 제2조제6호에 따른 국민주택 규모 이하 주...
77650,If the State and a local government sell any S...,법 제16조제6항에 따라 국가 및 지방자치단체가 개발사업시행자나 주요입주기업에 국유...
77676,The price in an ordinary transaction in any fr...,법 제21조에 따라 경제자유구역에서 건당 10만 달러 이하의 경상거래에 따른 대가는...
77741,Each person who has been granted permission to...,카지노업 허가를 받은 자는 영업개시 후 2년이 지난 날부터 30일 이내에 제20조의...
408573,The limit of payment or receipt of amount that...,소액해외송금업자가 취급할 수 있는 건당 지급 및 수령 범위는 각각 미화 5천달러를 ...
408782,Where a resident makes a direct payment of cos...,거주자가 건당 미화 1만달러 이하의 경상거래에 따른 대가를 외국환업무취급기관등을 통...


In [ ]:
dollor_index = df_samp.loc[df_samp.curHyp.str.contains(r"미[^☞]+달러")].index

d1, d2 = "미합중국화폐 ", "미화 "
for i in dollor_index:
    cur_hyp = df_samp.loc[i, "curHyp"]
    end=cur_hyp.find("달러")
    start = cur_hyp.find(d1)
    if start==-1:
        start = cur_hyp.find(d2)
        new_hyp = cur_hyp.replace(d2, "")
        new_hyp = new_hyp.replace("달러", " USD")
    else:
        new_hyp = cur_hyp.replace(d1, "")
        new_hyp = new_hyp.replace("달러", " USD")
    df_samp.loc[i, "curHyp"] = new_hyp
    df_samp.loc[i, "errors"].append({"severity":"neutral", "start_idx":start, "text":cur_hyp[start:end+2], "type":"mis-translation"})

In [ ]:
df_samp.loc[dollor_index].iloc[:, 3:]

,KOR,curHyp,errors
394528,경제자유구역에서의 관광사업에 투자하려는 외국인투자 금액이 미합중국화폐 5억달러 이상일 것,관광사업에 투자하려는 외국인투자 금액이 5억 USD 이상일 것,"[{'severity': 'major', 'start_idx': 0, 'text':..."
77741,카지노업 허가를 받은 자는 영업개시 후 2년이 지난 날부터 30일 이내에 제20조의...,카지노업 허가를 받은 자는 영업개시 후 2년이 지난 날부터 30일 이내에 제20조의...,"[{'severity': 'major', 'start_idx': 58, 'text'..."
408573,소액해외송금업자가 취급할 수 있는 건당 지급 및 수령 범위는 각각 미화 5천달러를 ...,소액해외송금업자가 취급할 수 있는 건당 지급 및 수령 범위는 각각 5천 USD를 한...,"[{'severity': 'neutral', 'start_idx': 37, 'tex..."
408782,거주자가 건당 미화 1만달러 이하의 경상거래에 따른 대가를 외국환업무취급기관등을 통...,거주자가 건당 1만 USD 이하의 경상거래에 따른 대가를 외국환업무취급기관등을 통하...,"[{'severity': 'neutral', 'start_idx': 8, 'text..."
408809,법 제20조제1항에 따라 비거주자에 대한 채권보유현황을 보고하여야 하는 대상은 미화...,법 제20조제1항에 따라 비거주자에 대한 채권보유현황을 보고하여야 하는 대상은 1만...,"[{'severity': 'neutral', 'start_idx': 44, 'tex..."
408915,"법 제29조제1항제4호에서 ""대통령령으로 정하는 금액""이란 미화 3만달러를 말한다.","법 제29조제1항제4호에서 ""대통령령으로 정하는 금액""이란 3만 USD를 말한다.","[{'severity': 'neutral', 'start_idx': 33, 'tex..."
417058,미화 100만달러 이상으로서 금융위원회가 정하여 고시하는 금액 이상의 자기자본을 갖출 것,100만 USD 이상으로서 금융위원회가 정하여 고시하는 금액 이상의 자기자본을 갖출 것,"[{'severity': 'neutral', 'start_idx': 0, 'text..."
417060,직전 사업연도말 현재 운용자산규모(금융위원회가 정하여 고시하는 방법에 따라 계산한 ...,직전 사업연도말 현재 운용자산규모(금융위원회가 정하여 고시하는 방법에 따라 계산한 ...,"[{'severity': 'neutral', 'start_idx': 55, 'tex..."
408781,계약 건당 미화 5만달러 이내의 수출대금을 기획재정부장관이 정하여 고시하는 기간을 ...,계약 건당 5만 USD 이내의 수출대금을 기획재정부장관이 정하여 고시하는 기간을 초...,"[{'severity': 'neutral', 'start_idx': 6, 'text..."


In [ ]:
percent_idx = df_samp.loc[df_samp.curHyp.str.contains("퍼센트")].iloc[:,3:].index

In [ ]:
df_samp.loc[df_samp.curHyp.str.contains("퍼센트")].iloc[:,3:]

,KOR,curHyp,errors
399090,관리기관이 법 제11조의 규정에 의하여 받는 보증료는 보증한 금액에 보증료율을 곱하...,관리기관이 법 제11조의 규정에 의하여 받는 보증료는 보증한 금액에 보증료율을 곱하...,[]
399094,관리기관이 법 제11조의2의 규정에 의하여 징수하는 위약금은 보증한 채무중 이행되지...,관리기관이 법 제11조의2의 규정에 의하여 징수하는 위약금은 보증한 채무중 이행되지...,[]
77650,법 제16조제6항에 따라 국가 및 지방자치단체가 개발사업시행자나 주요입주기업에 국유...,법 제16조제6항에 따라 국가 및 지방자치단체가 개발사업시행자나 주요입주기업에 국유...,"[{'severity': 'major', 'start_idx': 47, 'text'..."
519213,1세대무주택자의 경우: 보증금담보대출등을 받아 임차한 임차주택의 보증금 및 월세금액...,1세대무주택자의 경우: 보증금담보대출등을 받아 임차한 임차주택의 보증금 및 월세금액...,[]
519215,1세대무주택자의 경우: 보증금담보대출등 금액의 합산액(상환한 금액은 제외한다)의 3...,1세대무주택자의 경우: 보증금담보대출등 금액의 합산액(상환한 금액은 제외한다)의 3...,[]


In [ ]:
for i in percent_idx:
    cur_hyp = df_samp.loc[i, "curHyp"]
    start = cur_hyp.find("퍼센트")
    new_hyp = cur_hyp.replace("퍼센트", "%")
    df_samp.loc[i, "curHyp"] = new_hyp
    df_samp.loc[i, "errors"].append({"severity":"neutral", "start_idx":start, "text":"퍼센트", "type":"mis-translation"})
df_samp.loc[percent_idx]

,category,lawName,ENG,KOR,curHyp,errors
399090,통화ㆍ국채ㆍ금융,농림수산업자 신용보증법 시행령,The fees for guarantee which the management in...,관리기관이 법 제11조의 규정에 의하여 받는 보증료는 보증한 금액에 보증료율을 곱하...,관리기관이 법 제11조의 규정에 의하여 받는 보증료는 보증한 금액에 보증료율을 곱하...,"[{'severity': 'neutral', 'start_idx': 75, 'tex..."
399094,통화ㆍ국채ㆍ금융,농림수산업자 신용보증법 시행령,The penalties for breach of contract which the...,관리기관이 법 제11조의2의 규정에 의하여 징수하는 위약금은 보증한 채무중 이행되지...,관리기관이 법 제11조의2의 규정에 의하여 징수하는 위약금은 보증한 채무중 이행되지...,"[{'severity': 'neutral', 'start_idx': 58, 'tex..."
77650,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법 시행령,If the State and a local government sell any S...,법 제16조제6항에 따라 국가 및 지방자치단체가 개발사업시행자나 주요입주기업에 국유...,법 제16조제6항에 따라 국가 및 지방자치단체가 개발사업시행자나 주요입주기업에 국유...,"[{'severity': 'major', 'start_idx': 47, 'text'..."
519213,사회복지,국민건강보험법 시행령,In cases of a member of a household without ho...,1세대무주택자의 경우: 보증금담보대출등을 받아 임차한 임차주택의 보증금 및 월세금액...,1세대무주택자의 경우: 보증금담보대출등을 받아 임차한 임차주택의 보증금 및 월세금액...,"[{'severity': 'neutral', 'start_idx': 109, 'te..."
519215,사회복지,국민건강보험법 시행령,In cases of a member of a household without ho...,1세대무주택자의 경우: 보증금담보대출등 금액의 합산액(상환한 금액은 제외한다)의 3...,1세대무주택자의 경우: 보증금담보대출등 금액의 합산액(상환한 금액은 제외한다)의 3...,"[{'severity': 'neutral', 'start_idx': 47, 'tex..."


In [ ]:
df_samp.loc[df_samp.errors.apply(lambda x: len(x))==0].shape[0]

18576

##3.4.Major - Omission (법령명을 생략하는 경우)

In [ ]:
law_index = df_samp.loc[(df_samp.errors.apply(lambda x: len(x))==0)&(df_samp.KOR.str.contains("「"))].index

In [ ]:
for i in law_index:
    cur_hyp = df_samp.loc[i, "curHyp"]
    start = cur_hyp.find("「")
    end = cur_hyp.find("」")
    addition_str = ""
    try:
        while cur_hyp[end+1]!=" ":
            addition_str += cur_hyp[end+1]
            end+=1
    except:
        pass

    df_samp.loc[i, "curHyp"] = cur_hyp[:start].rstrip()+" "+cur_hyp[end+1:].lstrip()
    df_samp.loc[i, "errors"].append({"severity":"major", "start_idx":start, "text":cur_hyp[start:end+1].strip(), "type":"omission"})

In [ ]:
df_samp.loc[law_index]

,category,lawName,ENG,KOR,curHyp,errors
547121,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the average monthly remuneration of each...,"예술인[「고용보험법」 제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술...","예술인[ 제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술인""이라 한다...","[{'severity': 'major', 'start_idx': 4, 'text':..."
547123,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the amount of monthly remuneration of ea...,"노무제공자[「고용보험법」 제77조의6제2항제2호 단서에 따른 단기노무제공자(이하 ""...","노무제공자[ 제77조의6제2항제2호 단서에 따른 단기노무제공자(이하 ""단기노무제공자...","[{'severity': 'major', 'start_idx': 6, 'text':..."
394158,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,"The term ""alien"" means a person who falls unde...","""외국인""이란 「외국인투자 촉진법」 제2조제1항제1호에 해당하는 자를 말한다.","""외국인""이란 제2조제1항제1호에 해당하는 자를 말한다.","[{'severity': 'major', 'start_idx': 8, 'text':..."
394159,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,"The term ""foreign-invested enterprise"" means a...","""외국인투자기업""이란 「외국인투자 촉진법」 제2조제1항제6호에 따른 기업을 말한다.","""외국인투자기업""이란 제2조제1항제6호에 따른 기업을 말한다.","[{'severity': 'major', 'start_idx': 12, 'text'..."
394161,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,"The term ""repatriating enterprise"" means an en...","""국내복귀기업""이란 「해외진출기업의 국내복귀 지원에 관한 법률」 제7조에 따라 지원...","""국내복귀기업""이란 제7조에 따라 지원대상으로 선정된 기업을 말한다.","[{'severity': 'major', 'start_idx': 11, 'text'..."
...,...,...,...,...,...,...
194486,노동,산업재해보상보험법 시행령,Software engineers defined in subparagraph 10 ...,「소프트웨어 진흥법」 제2조제3호의 소프트웨어사업에서 노무를 제공하는 같은 조 제1...,제2조제3호의 소프트웨어사업에서 노무를 제공하는 같은 조 제10호에 따른 소프트웨...,"[{'severity': 'major', 'start_idx': 0, 'text':..."
194490,노동,산업재해보상보험법 시행령,Where the employer submits documents required ...,「관세법」 제222조제1항 및 같은 법 시행령 제231조에 따라 세관장에게 보세운송...,제222조제1항 및 같은 법 시행령 제231조에 따라 세관장에게 보세운송업자 등록...,"[{'severity': 'major', 'start_idx': 0, 'text':..."
194491,노동,산업재해보상보험법 시행령,Where the employer enters information on the o...,「물류정책기본법」 제29조의2제5항에 따라 위험물질운송차량의 소유자 정보 및 운전자...,제29조의2제5항에 따라 위험물질운송차량의 소유자 정보 및 운전자 정보 등을 위험...,"[{'severity': 'major', 'start_idx': 0, 'text':..."
194492,노동,산업재해보상보험법 시행령,Where the employer submits a plan to transport...,「화학물질관리법」제15조제3항에 따라 환경부장관에게 유해화학물질 운반계획서를 제출한 경우,따라 환경부장관에게 유해화학물질 운반계획서를 제출한 경우,"[{'severity': 'major', 'start_idx': 0, 'text':..."


In [ ]:
df_samp.loc[(df_samp.errors.apply(lambda x: len(x))==0)].iloc[:, 2:4]

,ENG,KOR
547124,"""Where any ground prescribed by Presidential D...","법 제3조제1항에서 ""대통령령으로 정하는 사유에 해당하는 경우""란 다음 각 호의 어..."
547125,The standard remuneration under Article 3 (2) ...,법 제3조제2항(법 제48조의2제8항제2호 및 제48조의3제8항제2호에서 준용하는 ...
547126,Where an artist falls under any of the followi...,예술인이 다음 각 호의 어느 하나에 해당하는 경우 법 제48조의2제8항제2호에서 준...
547127,Where a worker falls under any of the followin...,노무제공자가 다음 각 호의 어느 하나에 해당하는 경우 법 제48조의3제8항제2호에서...
394152,The purpose of this Act is to facilitate forei...,이 법은 경제자유구역의 지정 및 운영을 통하여 외국인투자기업 및 국내복귀기업의 경영...
...,...,...
78001,The right to receive payments of principal and...,국채의 원금 및 이자를 받을 권리는 5년간 행사하지 아니하면 시효(시효)의 완성으로...
78006,Any person requested to provide materials rela...,제1항에 따라 국채 관련 자료의 제출을 요청받은 자는 특별한 사유가 없으면 요청에 ...
78007,Where necessary to formulate and carry out pol...,기획재정부장관은 국채시장에 관한 정책의 수립·운영에 필요한 경우에는 관계 중앙행정기...
406361,"Notwithstanding paragraph (3), a workplace cre...",제3항의 규정에 불구하고 제12조제1항제2호의 직장조합 및 동항제3호 라목의 단체조...


## 나중에

In [ ]:
pat = r"\"[ㄱ-ㅎ|가-힣]+\""
quote_comp = re.compile(pat)
quote_words = df_samp.loc[df_samp.KOR.str.contains(pat)].KOR.apply(lambda x: quote_comp.findall(x))
quote_words

547121                    ["단기예술인"]
547123                  ["단기노무제공자"]
394156                   ["경제자유구역"]
394158                      ["외국인"]
394159                  ["외국인투자기업"]
                    ...            
194419    ["위원장", "재심사위원회", "소위원회"]
194455             ["진찰", "특진의료기관"]
416399        ["의결권권유자", "의결권피권유자"]
194470             ["근로자", "진폐근로자"]
194496         ["근로자", "특수형태근로종사자"]
Name: KOR, Length: 1716, dtype: object

In [ ]:
quword4 = quote_words[quote_words.apply(lambda x: max(list(map(lambda k: len(k), x)))) > 8]

In [ ]:
qw_srs = quword4.apply(lambda x: sorted(x, key=lambda k: len(k), reverse=True)[0].strip('"').rstrip("등"))
sort_index = qw_srs.apply(len).sort_values().index
qw_srs[sort_index]

548555                출산전후급여
81365                 대부중개업자
397510                은행지주회사
207755                지정의료기관
83487                 여신금융기관
                 ...        
422493          주택담보노후연금보증채무
418500         금융투자상품거래청산업인가
416776        외국금융투자상품거래청산회사
556053        산업재해보상보험재심사위원회
519095    부당이득징수금체납정보공개심의위원회
Name: KOR, Length: 409, dtype: object

In [ ]:
origin_text = list(set(list(qw_srs.values)))
origin_text
# origin_text

## 다른 오류 생성

In [ ]:
df_samp.loc[df_samp.errors.apply(len)==0].shape[0]

3903

In [ ]:
not_yet_index = df_samp.loc[df_samp.errors.apply(len)==0].index

In [ ]:
temp = df_samp.loc[not_yet_index]
temp.loc[temp.curHyp.str.contains("할 수 있다")].iloc[:, 2:4]

In [ ]:
pattern = r"[\d]+조[^\항]*[\d]+항"
hangho = re.compile(pattern)
txtx = "집합투자업자는 제80조제4항 및 제86조제1항에도 불구하고 법 제81조제1항 단서 및 제234조제4항에 따라 상장지수집합투자기구(투자자 보호 등을 고려하여 금융위원회가 정하여 고시하는 상장지수집합투자기구에 한정한다)의 집합투자재산을 다음 각 호의 방법으로 운용할 수 있다."
sres = hangho.search(txtx)
print(sres)
fnum = txtx[sres.span()[0]]
snum = txtx[sres.span()[1]-2]
print(fnum, snum)

<re.Match object; span=(9, 15), match='80조제4항'>
8 4


In [ ]:
def make_dict(df):
    output_json = dict()
    output_json["data"] = []
    for i in df.index:
        base_form = {"category": None, "en": None, "errs": None, "hyp": None, "ko": None, "law_name": None}
        if len(df.loc[i, "errors"])==0:
            continue
        base_form["category"] = df.loc[i, "category"]
        base_form["en"] = df.loc[i, "ENG"]
        base_form["errs"] = df.loc[i, "errors"]
        base_form["hyp"] = df.loc[i, "curHyp"]
        base_form["ko"] = df.loc[i, "KOR"]
        base_form["law_name"] = df.loc[i, "lawName"]
        output_json["data"].append(base_form)
    return output_json

rrrr = make_dict(df_samp)

In [ ]:
with open(os.path.join(gdrive_path, "data_0823.json"), "w") as f:
  json.dump(rrrr, f, ensure_ascii=False, indent=4)